<a href="https://colab.research.google.com/github/nedokormysh/GB_PyTorch/blob/lesson3/GB_PyTorch_hw_3_dirty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Будем практиковаться на датасете недвижимости (sklearn.datasets.fetch_california_housing)

Ваша задача:

Создать Dataset для загрузки данных
Обернуть его в Dataloader
Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели
train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import torch.nn.functional as F
from torch import nn
from torch import optim
import torchvision
import torchvision.transforms as transforms

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

In [ ]:
# dataset = fetch_california_housing(return_X_y=False)

# data = fetch_california_housing()

# df = pd.DataFrame(data['data'])
# df['target'] = data['target']

# df.info()

In [ ]:
data = fetch_california_housing()

In [ ]:
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

In [ ]:
data.DESCR

'.. _california_housing_dataset:\n\nCalifornia Housing dataset\n--------------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 20640\n\n    :Number of Attributes: 8 numeric, predictive attributes and the target\n\n    :Attribute Information:\n        - MedInc        median income in block group\n        - HouseAge      median house age in block group\n        - AveRooms      average number of rooms per household\n        - AveBedrms     average number of bedrooms per household\n        - Population    block group population\n        - AveOccup      average number of household members\n        - Latitude      block group latitude\n        - Longitude     block group longitude\n\n    :Missing Attribute Values: None\n\nThis dataset was obtained from the StatLib repository.\nhttps://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html\n\nThe target variable is the median house value for California districts,\nexpressed in hundreds of thousands of dollars ($100,000

In [ ]:
df = pd.DataFrame(data.data, columns=data.feature_names)
# Добавьте столбец "target" и заполните его данными.
df['target'] = data.target
# Посмотрим первые пять строк
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      20640 non-null  float64
 1   HouseAge    20640 non-null  float64
 2   AveRooms    20640 non-null  float64
 3   AveBedrms   20640 non-null  float64
 4   Population  20640 non-null  float64
 5   AveOccup    20640 non-null  float64
 6   Latitude    20640 non-null  float64
 7   Longitude   20640 non-null  float64
 8   target      20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


Нет пропусков и нет строковых признаков.

In [ ]:
df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


In [ ]:
import math
import torch
import numpy as np

from sklearn.preprocessing import StandardScaler

class Clfrndataset(torch.utils.data.Dataset):
    def __init__(self, train=False):
      X, y = fetch_california_housing(return_X_y=True)
      scaler = StandardScaler()
      X = scaler.fit_transform(X)
      X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13, 
                                                                test_size=0.25)

      # self.data = X
      # self.labels = y
      # т.к. попросили реализовать разбиение в самом классе, то нам нужно знать 
      # что мы подаём на вход нашего класса - train или test

      if train:
          self.data = X_train
          self.labels = y_train
      else:
          self.data = X_test
          self.labels = y_test

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index, :], self.labels[index]

In [ ]:
X, y = fetch_california_housing(return_X_y=True) # если return = True, то возвращаем не словарями, а двумя таблицами - вектор ответов и матрица признаков

In [ ]:
X[0, :]

array([   8.3252    ,   41.        ,    6.98412698,    1.02380952,
        322.        ,    2.55555556,   37.88      , -122.23      ])

In [ ]:
X[:, 0]

array([8.3252, 8.3014, 7.2574, ..., 1.7   , 1.8672, 2.3886])

In [ ]:
X

array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]])

In [ ]:
train_dataset = Clfrndataset(train=True)
test_dataset = Clfrndataset()


train_data_loader = torch.utils.data.DataLoader(train_dataset, 
                                                batch_size=32, 
                                                shuffle=True, 
                                                num_workers=1)

test_data_loader = torch.utils.data.DataLoader(test_dataset, 
                                               batch_size=8,
                                               num_workers=1)

In [ ]:
class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu", device=None):
        super().__init__()
        self.fc = nn.Linear(input_dim, output_dim, device=device)
        self.activation = activation
        
    def forward(self, x):
        x = self.fc(x)
        if self.activation == "relu":
            return F.relu(x)
        if self.activation == "sigmoid":
            return torch.sigmoid(x)
        if self.activation == "elu":
            return F.elu(x)
        if self.activation == "tanh":
            return torch.tanh(x)
        raise RuntimeError